In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np

from datasets import get_dataset
from configs.get_configs import get_config
import torchaudio

In [27]:
import run_lib

In [45]:
class DotDict(dict):
    def __getattr__(self, key):
        return self[key]
    def __setattr__(self, key, val):
        if key in self.__dict__:
            self.__dict__[key] = val
        else:
            self[key] = val

### Initialize Experiment

In [56]:
args = DotDict()
args.conf = "64_deep"
args.test = False
args.config = get_config(args)
args.workdir = "64_deep"
args.eval_folder = "base"

Read Config: 
data:
  category: mel
  centered: false
  channels: 1
  dataset: speech_commands
  image_height: 64
  image_width: 64
  mel_root: mel_datasets/sc09_64
  num_channels: 1
  spec:
    audio_length: 1
    hop_length: 256
    image_size: 64
    nfft: 1024
    num_mels: 64
    sample_rate: 16000
    spec_len_samples: 64
  uniform_dequantization: false
device: !!python/object/apply:torch.device
- cpu
eval:
  batch_size: 32
  begin_ckpt: 30
  bpd_dataset: test
  enable_bpd: false
  enable_interpolate: false
  enable_loss: false
  enable_rescale: false
  enable_sampling: true
  end_ckpt: 31
  num_samples: 32
  save_images: true
model:
  attention_type: ddpm
  attn_resolutions: !!python/tuple
  - 16
  beta_max: 20.0
  beta_min: 0.1
  ch_mult: !!python/tuple
  - 1
  - 1
  - 2
  - 2
  - 4
  - 4
  conv_size: 3
  dropout: 0.0
  ema_rate: 0.995
  embedding_type: positional
  fourier_scale: 16
  init_scale: 0.0
  name: ncsnpp_audio
  nf: 128
  nonlinearity: swish
  normalization: GroupNo

In [58]:
run_lib.evaluate(args, args.workdir, args.eval_folder)

AttributeError: "'config'"

### Sampling Hyperparameter Search and Score Computation

KeyError: 'eval'

### Generate Samples for best Hyperparameters